In [1]:
# This file should be contained in the same folder with the initial data set folder

# Used in Data Preparation
import os # Used to create folders
from os import listdir # Used to access images folders
from os.path import isfile, join # Used to access images files
import numpy as np
import pandas as pd
from itertools import product
from PIL import Image # Used to read images
from keras.preprocessing.image import ImageDataGenerator # Used to populate images


Using TensorFlow backend.


In [ ]:
if __name__ == '_main__':
    copies_per_image = 4
    np.random.seed(123)

    count = 0
    #initialize a dataframe for the count of train, validate and test images
    sets_counter = pd.DataFrame([[0, 0, 0]], columns=['train', 'valid', 'test'], index=['counter'])
    #the ImageDataGenerator accepts the original data, randomly transforms it, and returns only the new, transformed data.
    datagen = ImageDataGenerator(featurewise_center=False,
                                 featurewise_std_normalization=False,
                                 rotation_range=20,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True)
    
    #make the below folders in the below paths
    os.mkdir('augmented_dataset')
    os.mkdir('augmented_dataset/train')
    os.mkdir('augmented_dataset/train/benign')
    os.mkdir('augmented_dataset/train/malignant')
    os.mkdir('augmented_dataset/valid')
    os.mkdir('augmented_dataset/valid/benign')
    os.mkdir('augmented_dataset/valid/malignant')
    os.mkdir('augmented_dataset/test')
    os.mkdir('augmented_dataset/test/test_images')

    #for every image 
    for a, b in product(['train', 'test'], ['benign', 'malignant']):
        folder_path = 'data/' + a + '/' + b
    
        for f in listdir(folder_path):
            image_path = join(folder_path, f)
        
            # if the path exists
            if isfile(image_path):
                count += 1
                # convert each image into RGB triplette  array
                im = np.expand_dims(Image.open(image_path).convert("RGB"), 0)
                datagen.fit(im)
            
                # split the image in train (70 %), validate (10%), test(20 %) based on r value (produced uniformly)
                for _ in range(copies_per_image):
                    save_to_dir = 'augmented_dataset/'
                    r = np.random.uniform()
            
                    if r < 0.1:
                        sets_counter['valid'] += 1
                        save_to_dir = join(save_to_dir, 'valid', b)
                    elif r < 0.3:
                        sets_counter['test'] += 1
                        save_to_dir = join(save_to_dir, 'test/test_images')
                    else:
                        sets_counter['train'] += 1
                        save_to_dir = join(save_to_dir, 'train', b)
            
                    for x in datagen.flow(im,
                                          save_to_dir=save_to_dir,
                                          save_prefix=b + '_' + str(count),
                                          save_format='jpg'):
                        break # Only one picture
            
                if count % 1000 == 0:
                    print(count, 'images were processed')
    #print the number of images processed           
    print(count, 'images were processed')
    #print the dataframe containing the images per train, validate and test
    print(sets_counter)
